In [1]:
#Fitting a model for article 6

In [100]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 20)  
pd.set_option('display.max_colwidth', -1) 

import ast
import math
import random
import html2text
from IPython.core.display import display, HTML

import nltk
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize

import functions 
clean = functions.clean()


In [201]:
attributes = pd.read_csv("/Users/conorosully/Documents/Legal-Case-Prediction/data/preclean/case_attributes.csv",'#')
print(len(attributes))
print(attributes.columns)
attributes.head(3)

65037
Index(['id', 'type', 'url', 'doc-name', 'doc-id-label', 'doc-date', 'vl-type',
       'created-at', 'updated-at', 'app-no-parts', 'app-nos', 'articles',
       'conclusions', 'courts', 'ecli', 'judgement-date', 'judges',
       'last-modified-time', 'nonviolations', 'organisations', 'respondents',
       'separate-opinion', 'violations'],
      dtype='object')


,id,type,url,doc-name,doc-id-label,doc-date,vl-type,created-at,updated-at,app-no-parts,app-nos,articles,conclusions,courts,ecli,judgement-date,judges,last-modified-time,nonviolations,organisations,respondents,separate-opinion,violations
0,HUDOC-ECHR-1956-001-142534,eu-chrs,http://hudoc.echr.coe.int/app/conversion/docx/html/body?library=ECHR&id=001-142534,greece <br>v.<br> the united kingdom,176/56,1956-06-02,eu_chr,2017-09-18T10:01:36.116Z,2018-10-26T16:53:54.496Z,"['176', '56']",['176/56'],"['33', '35']",['Admissible'],NaN,ECLI:CE:ECHR:1956:0602DEC000017656,NaN,NaN,2018-10-25,NaN,NaN,['GBR'],NaN,NaN
1,HUDOC-ECHR-1957-001-110191,eu-chrs,http://hudoc.echr.coe.int/app/conversion/docx/html/body?library=ECHR&id=001-110191,decision by the commission on the admissibility,250/57,1957-07-20,eu_chr,2017-09-18T10:01:36.107Z,2018-10-26T16:53:51.559Z,"['250', '57']",['250/57'],"['9', '10', '11', '35']",['Inadmissible'],"['Constitutional Court', 'Federal Constitutional Court']",ECLI:CE:ECHR:1957:0720DEC000025057,NaN,NaN,2018-10-08,NaN,NaN,['DEU'],NaN,NaN
2,HUDOC-ECHR-1957-001-170091,eu-chrs,http://hudoc.echr.coe.int/app/conversion/docx/html/body?library=ECHR&id=001-170091,x. <br>v.<br> sweden,172/56,1957-12-20,eu_chr,2017-09-18T10:01:36.121Z,2018-10-26T16:53:51.867Z,"['172', '56']",['172/56'],"['6', '8', '9', '35']",['Inadmissible'],"['Bundesverfassungsgericht', 'Constitutional Court', 'Federal Constitutional Court', 'Supreme Court']",ECLI:CE:ECHR:1957:1220DEC000017256,NaN,NaN,2018-10-08,NaN,NaN,['SWE'],NaN,NaN


In [31]:
#Find list of unique article codes

articles = [item for sublist in list(attributes['articles'])  for item in ast.literal_eval(sublist)]
#ast.literal_eval(list(att['articles'])[0])
unique_articles = list(pd.Series(articles).unique())
unique_articles

['33',
 '35',
 '9',
 '10',
 '11',
 '6',
 '8',
 '15',
 '15-1',
 '15-3',
 'P1-1',
 '8-2',
 '8-1',
 '9-1',
 '10-1',
 '11-1',
 'P1-1-1',
 'P7-1',
 'P7-1-1',
 '6-1',
 '6-2',
 '13',
 '5',
 '5-1',
 '1',
 'P1-2',
 '5-1-c',
 '5-2',
 '5-3',
 '6-3',
 '5-5',
 '7',
 '7-1',
 '17',
 '6-3-d',
 '14',
 '6-3-c',
 '3',
 '35-1',
 '35-3',
 '6-3-a',
 '5-1-b',
 '10-2',
 '2',
 '4',
 '6-3-b',
 '5-1-a',
 '5-4',
 '34',
 '53',
 '5-1-f',
 '31',
 '31-3',
 '25',
 '25-1',
 '57',
 '5-1-e',
 '6-3-e',
 '12',
 '35-2',
 '37',
 '37-1-a',
 '27',
 '27-2',
 '37-1',
 '18',
 'P1-3',
 '16',
 '9-2',
 '19',
 '32',
 '32-2',
 'P4-2',
 'P4-2-2',
 '4-2',
 '4-3-a',
 '4-1',
 '11-2',
 '44',
 '41',
 '14+P1-2',
 '30',
 '30-1-a',
 '30-1-b',
 '37-1-b',
 'P7-3',
 'P4-3',
 'P4-3-1',
 'P4-2-3',
 '4-3-b',
 'P4-1',
 'P1-1-2',
 '2-1',
 '15+5',
 '15+6',
 '28',
 '29',
 'P4-4',
 '14+P1-1',
 '14+8',
 '14+13',
 '14+5',
 '14+11',
 '14+6-1',
 '14+6-3-a',
 '14+6-3-b',
 '14+6-3-c',
 '14+6-3-e',
 '5-1-d',
 '56',
 '14+1',
 '14+17',
 '14+2',
 '14+3',
 '14+4',


In [32]:
#Find list of article codes that include article 6
article_6 = set() #List of codes relating to article 6
for u in unique_articles:
    sub_articles = u.split("+")
    for s in sub_articles:
        if s[0] == '6':
            article_6.add(u)
print(len(article_6))
list(article_6)[0:10]

65


['13+6-3-d',
 '7+6',
 '6-3-c+13',
 '14+6-3-e',
 '6-1+6-3-b',
 '6+6-3-d',
 '6-1+6-3-a',
 '6+6',
 '6+8',
 '6-2+13']

In [56]:
#Determin if a case has a complaint relating to article 6

complaint = [] #true if complaint, false otherwise
for articles in attributes['articles']:
    c = False
    for article in ast.literal_eval(articles):
        #Atleast one of the complaints must relate to Article 6
        if article in article_6: 
            c = True
            break;
    complaint.append(c)
print(len(complaint))
print(sum(complaint))

65037
36475


In [62]:
#Find all case ids that have a complaint relating to article 6
attributes_6 = attributes[complaint]
print(len(attributes_6))
print(list((attributes_6['articles'])[0:10]))
attributes_6.head(3)

36475
["['6', '8', '9', '35']", "['8', '8-2', '8-1', '9', '9-1', '10', '10-1', '11', '11-1', '15', '15-1', '15-3', 'P1-1', 'P1-1-1', 'P7-1', 'P7-1-1', '6', '6-1', '6-2', '13', '5', '5-1', '1']", "['6', '6-1', '8', '9', '35', 'P1-2']", "['5', '5-1-c', '5-2', '5-3', '6', '6-1', '6-2', '6-3', '35']", "['6', '6-1', '6-2', '6-3-d', '14', '33', '35']", "['6', '8', '9', '10', '14', '35']", "['6', '6-1']", "['6', '6-1', '6-3-c']", "['6', '6-1', '35', '35-3']", "['6', '6-1', '6-3-a']"]


,id,type,url,doc-name,doc-id-label,doc-date,vl-type,created-at,updated-at,app-no-parts,...,courts,ecli,judgement-date,judges,last-modified-time,nonviolations,organisations,respondents,separate-opinion,violations
2,HUDOC-ECHR-1957-001-170091,eu-chrs,http://hudoc.echr.coe.int/app/conversion/docx/...,x. <br>v.<br> sweden,172/56,1957-12-20,eu_chr,2017-09-18T10:01:36.121Z,2018-10-26T16:53:51.867Z,"['172', '56']",...,"['Bundesverfassungsgericht', 'Constitutional C...",ECLI:CE:ECHR:1957:1220DEC000017256,NaN,NaN,2018-10-08,NaN,NaN,['SWE'],NaN,NaN
5,HUDOC-ECHR-1958-001-73860,eu-chrs,http://hudoc.echr.coe.int/app/conversion/docx/...,greece <br>v.<br> the united kingdom (volume ii),176/56,1958-09-26,eu_chr,2017-09-18T10:01:36.133Z,2018-10-26T16:54:03.876Z,"['176', '56']",...,['Supreme Court'],ECLI:CE:ECHR:1958:0926REP000017656,NaN,NaN,2018-10-07,"['15', '15-1', '8', '5', '5-1', '1', '6', '6-2...","['International Court of Justice', 'Advisory C...",['GBR'],NaN,NaN
6,HUDOC-ECHR-1959-001-117006,eu-chrs,http://hudoc.echr.coe.int/app/conversion/docx/...,x. <br>v.<br> sweden,434/58,1959-06-30,eu_chr,2017-09-18T10:01:36.125Z,2018-10-26T16:53:54.341Z,"['434', '58']",...,NaN,ECLI:CE:ECHR:1959:0630DEC000043458,NaN,NaN,2018-10-08,NaN,NaN,['SWE'],NaN,NaN


In [76]:
list(attributes_6['conclusions'])[1]

'[\'No violation Article 15-1 as regards the application of the curfews law\', \'No violation as regards the regulations on detention without warrants Article 15-3\', \'No violation Art. 15-1 the detention of persons without trial under the Detention of Persons Law (strictly required by the exigencies of the situation\', \'No violation Art. 8 and Art. 5-1 as regards the regulations on deportation\', \'No violation Arts. 8, 9, 10 and 11 as regards the regulations on entry and search of premises with warrant, control over burials, censorship of correspondence, the press and speeches, illegal strikes, prohibition of public processions, meetings and assemblies, closing of schools, the jamming of broadcasts\', \'No violation of Art. 1 of Protocol No. 1\', \'No violation Art. 6 as regards Regulation of The Emergency Powers (Public Safety and Order) Regulations providing for the destruction of a building and its content\', \'No violation Art. 6-2 as regards the regulations imposing death\', \

In [125]:
#Investigate violations/ non-violations
print(attributes_6['violations'][0:10])
print(attributes_6['nonviolations'][0:10])
print(list(attributes_6['nonviolations'])[1])

2     NaN
5     NaN
6     NaN
7     NaN
13    NaN
14    NaN
15    NaN
16    NaN
18    NaN
20    NaN
Name: violations, dtype: object
2                                                   NaN
5     ['15', '15-1', '8', '5', '5-1', '1', '6', '6-2...
6                                                   NaN
7                                                   NaN
13                                                  NaN
14                                                  NaN
15                                                  NaN
16                                                  NaN
18                                                  NaN
20                                                  NaN
Name: nonviolations, dtype: object
['15', '15-1', '8', '5', '5-1', '1', '6', '6-2', '13']


In [127]:
#Determine if in violation, non-violation, or other of article 6 

violation_6 = [] #true if at least one Article 6 complaint = violation
for violations in attributes_6['violations']:
    
    v = False
    
    if type(violations) != float:
        for violation in ast.literal_eval(violations):
            #Atleast one of the complaint violations must relate to Article 6
            if violation in article_6: 
                v = True
                break;
    
    violation_6.append(v)
    
print(len(violation_6))
print(sum(violation_6))
attributes_6[violation_6]['violations'][0:10]

36475
13239


83                                                 ['6']
235    ['3', '5', '6', '8', '9', '10', '11', '13', '14']
326                                           ['6', '8']
378                                    ['6', '6-1', '8']
435                             ['5', '5-1', '6', '6-1']
524                                         ['6', '6-1']
617                                         ['6', '6-1']
618                                       ['6', '6-3-e']
623                                                ['6']
699                                    ['6', '6-1', '8']
Name: violations, dtype: object

In [128]:
# Same as above but for non-violations
nonviolation_6 = [] #true if at least one Article 6 complaint = violation
for nonviolations in attributes_6['nonviolations']:
    
    nv = False
    
    if type(nonviolations) != float:
        for nonviolation in ast.literal_eval(nonviolations):
            #Atleast one of the complaint violations must relate to Article 6
            if nonviolation in article_6: 
                nv = True
                break;
    
    nonviolation_6.append(nv)
    
print(len(nonviolation_6))
print(sum(nonviolation_6))
attributes_6[nonviolation_6]['nonviolations'][0:10]

36475
1279


5      ['15', '15-1', '8', '5', '5-1', '1', '6', '6-2...
23                                                 ['6']
31                                                 ['6']
58                                                 ['6']
84                            ['5', '5-1-c', '6', '6-1']
162                             ['5', '5-3', '6', '6-1']
193                      ['5', '5-3', '5-4', '6', '6-1']
236                                         ['6', '6-1']
238                                                ['6']
239                                                ['6']
Name: nonviolations, dtype: object

In [138]:
# determin target from above 
# - If altleast one violation then target = 'violation'
target = []
for v,n in zip(violation_6,nonviolation_6):
    if v == True:
        target.append('violation')
    elif (v == False) & (n == True):
        target.append('nonviolation')
    else:
        target.append('other')
print(len(target))
pd.Series(target).value_counts()

36475


other           22273
violation       13239
nonviolation      963
dtype: int64

In [148]:
# Find those cases with only violation/ non-violation outcomes 
attributes_6['target']  = target
vn_attributes_6 = attributes_6[(attributes_6['target'].isin(['violation','nonviolation']))] 
print(len(vn_attributes_6))


14202


/Users/conorosully/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [153]:
# Look at courts
courts = [item for sublist in list(vn_attributes_6['courts'].dropna())  for item in ast.literal_eval(sublist)]
#ast.literal_eval(list(att['articles'])[0])
pd.Series(courts).value_counts().head()

Supreme Court                   3143
Constitutional Court            1811
Court of Cassation              1289
Administrative Court             961
Supreme Administrative Court     396
dtype: int64

In [155]:
attributes_6.columns

Index(['id', 'type', 'url', 'doc-name', 'doc-id-label', 'doc-date', 'vl-type',
       'created-at', 'updated-at', 'app-no-parts', 'app-nos', 'articles',
       'conclusions', 'courts', 'ecli', 'judgement-date', 'judges',
       'last-modified-time', 'nonviolations', 'organisations', 'respondents',
       'separate-opinion', 'violations', 'target'],
      dtype='object')

Weird:
HUDOC-ECHR-1992-002-9778
HUDOC-ECHR-1999-001-58355 - 
HUDOC-ECHR-1999-001-50303 - 
HUDOC-ECHR-1995-001-49459
HUDOC-ECHR-2007-001-83755
HUDOC-ECHR-2000-001-58681
HUDOC-ECHR-2002-001-56046
HUDOC-ECHR-2015-001-152946

Normal:
HUDOC-ECHR-2001-001-5758
HUDOC-ECHR-2017-001-176245

In [204]:
#Look at weird cases:
ID = [
    "HUDOC-ECHR-1992-002-9778",
"HUDOC-ECHR-1999-001-58355",
"HUDOC-ECHR-1999-001-50303",
"HUDOC-ECHR-1995-001-49459",
"HUDOC-ECHR-2007-001-83755",
"HUDOC-ECHR-2000-001-58681",
"HUDOC-ECHR-2002-001-56046",
"HUDOC-ECHR-2015-001-152946"
]

attributes_6[attributes_6['id'].isin(ID)]

,id,type,url,doc-name,doc-id-label,doc-date,vl-type,created-at,updated-at,app-no-parts,app-nos,articles,conclusions,courts,ecli,judgement-date,judges,last-modified-time,nonviolations,organisations,respondents,separate-opinion,violations,target
3823,HUDOC-ECHR-1992-002-9778,eu-chrs,http://hudoc.echr.coe.int/app/conversion/docx/html/body?library=ECHR&id=002-9778,cormio <br>v.<br> italy,13130/87,1992-02-27,eu_chr,2017-09-18T12:05:13.797Z,2018-10-26T18:36:33.358Z,"['13130', '87']",['13130/87'],"['6', '6-1']",['No violation of Art. 6-1'],NaN,NaN,NaN,NaN,2018-10-08,NaN,['ECHR'],['ITA'],NaN,NaN,other
5755,HUDOC-ECHR-1995-001-49459,eu-chrs,http://hudoc.echr.coe.int/app/conversion/docx/html/body?library=ECHR&id=001-49459,tripodi and ficara <br>against<br> italy,15486/89,1995-09-11,eu_chr,2017-09-18T12:37:59.557Z,2018-10-26T19:52:36.111Z,"['15486', '89']",['15486/89'],"['6', '6-1']","['Violation of Art. 6-1', 'Information given by the government concerning measures taken to prevent new violations. Payment of the sums provided for in the decision of the Committee of Ministers.']",NaN,NaN,NaN,NaN,2018-10-07,NaN,"['Committee of Ministers', 'ECHR']",['ITA'],NaN,"['6', '6-1']",violation
11812,HUDOC-ECHR-1999-001-50303,eu-chrs,http://hudoc.echr.coe.int/app/conversion/docx/html/body?library=ECHR&id=001-50303,f. d. s. <br>against<br> france,33848/96,1999-07-15,eu_chr,2017-09-18T16:11:23.478Z,2018-10-26T23:06:11.494Z,"['33848', '96']",['33848/96'],"['6', '6-1']",['Violation of Art. 6-1'],NaN,NaN,NaN,NaN,2018-10-07,NaN,"['Committee of Ministers', 'ECHR']",['FRA'],NaN,"['6', '6-1']",violation
12258,HUDOC-ECHR-1999-001-58355,eu-chrs,http://hudoc.echr.coe.int/app/conversion/docx/html/body?library=ECHR&id=001-58355,gros <br>v.<br> france,43743/98,1999-11-09,eu_chr,2017-09-18T16:11:41.504Z,2018-10-26T23:21:38.872Z,"['43743', '98']",['43743/98'],"['6', '6-1', '37', '37-1', '38', '38-1-b', '39']",['Struck out of the list (friendly settlement)'],NaN,ECLI:CE:ECHR:1999:1109JUD004374398,1999-11-09,NaN,2018-10-07,NaN,NaN,['FRA'],False,NaN,other
13474,HUDOC-ECHR-2000-001-58681,eu-chrs,http://hudoc.echr.coe.int/app/conversion/docx/html/body?library=ECHR&id=001-58681,grasso <br>v.<br> italy,43074/98,2000-06-22,eu_chr,2017-09-18T16:17:46.884Z,2018-10-26T23:51:34.320Z,"['43074', '98']",['43074/98'],"['6', '6-1', '37', '37-1', '38', '38-1-b', '39']",['Struck out of the list (friendly settlement)'],NaN,ECLI:CE:ECHR:2000:0622JUD004307498,2000-06-22,NaN,2018-10-07,NaN,NaN,['ITA'],False,NaN,other
16456,HUDOC-ECHR-2002-001-56046,eu-chrs,http://hudoc.echr.coe.int/app/conversion/docx/html/body?library=ECHR&id=001-56046,aït-mouhoub <br>against<br> france,22924/93,2002-03-06,eu_chr,2017-09-18T16:27:13.798Z,2018-10-27T01:26:40.657Z,"['22924', '93']",['22924/93'],"['6', '6-1']",['Information given by the government concerning measures taken to prevent new violations. Payment of the sums provided for in the judgment.'],['Cour de cassation'],NaN,1998-10-28,NaN,2018-10-21,NaN,"['Committee of Ministers', 'ECHR']",['FRA'],NaN,NaN,other
28011,HUDOC-ECHR-2007-001-83755,eu-chrs,http://hudoc.echr.coe.int/app/conversion/docx/html/body?library=ECHR&id=001-83755,denee <br>v.<br> belgium,31634/03,2007-12-04,eu_chr,2017-09-19T06:13:27.481Z,2018-10-27T09:12:52.098Z,"['31634', '03']",['31634/03'],"['6', '6-1']",['Violation of Art. 6-1'],NaN,ECLI:CE:ECHR:2007:1204JUD003163403,2007-12-04,NaN,2018-10-08,NaN,NaN,['BEL'],False,"['6', '6-1']",violation
52753,HUDOC-ECHR-2015-001-152946,eu-chrs,http://hudoc.echr.coe.int/app/conversion/docx/html/body?library=ECHR&id=001-152946,parazajder <br>v.<br> croatia,50049/12,2015-02-16,eu_chr,2017-09-19T08:31:16.695Z,2018-10-27T23:58:21.662Z,"['50049', '12']",['50049/12'],"['6', '6-1', '8', '8-1']","['Communicated', 'Communicated']","['Constitutional Court', 'Supreme Court', 'Ustavni Sud Republike Hrvatske']",NaN,NaN,NaN,2018-10-08,NaN,NaN,['HRV'],NaN,NaN,other


In [206]:
# Look at structure of those cases
ID = list(vn_attributes_6['id'])
random_ID = random.sample(ID,1)[0]
random_ID = "HUDOC-ECHR-1995-001-49459"
clean.renderHTML(random_ID)

CASE: HUDOC-ECHR-1995-001-49459



In [208]:
# Save dataset up until this point
vn_attributes_6.to_csv('/Users/conorosully/Documents/Legal-Case-Prediction/data/test/A6_attributes.csv')

In [2]:
# Get final data
A6_attributes = pd.read_csv('/Users/conorosully/Documents/Legal-Case-Prediction/data/test/A6_attributes.csv',usecols=['id','target'])
A6_attributes.head()

,id,target
0,HUDOC-ECHR-1958-001-73860,nonviolation
1,HUDOC-ECHR-1961-001-73439,nonviolation
2,HUDOC-ECHR-1962-001-73435,nonviolation
3,HUDOC-ECHR-1964-001-73440,nonviolation
4,HUDOC-ECHR-1966-001-73425,violation


In [36]:
# Extract law section from each case
ID = list(A6_attributes['id'])
random_ID = random.sample(ID,1)[0]
print(A6_attributes[A6_attributes['id']==random_ID])
print(clean.getCase(random_ID))
clean.renderHTML(random_ID)

                               id     target
11785  HUDOC-ECHR-2012-001-112278  violation
{'HUDOC-ECHR-2012-001-112278': '<div>\n<p align="center">FIFTH SECTION</p>\n<p align="center"></p>\n<p align="center"></p>\n<p align="center"><b>CASE OF </b><b>JAMA v. SLOVENIA</b></p>\n<p align="center"></p>\n<p align="center"><i>(Application no. 48163/08)</i></p>\n<p align="center"></p>\n<p align="center"></p>\n<p align="center">JUDGMENT</p>\n<p align="center"></p>\n<p align="center"></p>\n<p align="center">STRASBOURG</p>\n<p align="center"></p>\n<p align="center">19 July 2012</p>\n<p align="center"></p>\n<p align="center"><b><u>FINAL</u></b></p>\n<p align="center"></p>\n<p align="center"><i>19/10/2012</i></p>\n<p align="center"></p>\n<p class="sub-section">This judgment has become final under Article 44 § 2 of the Convention. It may be subject to editorial revision.</p>\n</div>\n<br><div>\n<p class="section">In the case of Jama v. Slovenia,</p>\n<p>The European Court of Human Rights (Fifth Sect

In [3]:
def cleanFacts(ID):
    case = clean.getCase(ID)[ID]
    facts_raw = case.split('<h3>THE FACTS</h3>')[1].split("<h3>COMPLAINTS</h3>")[0].split('<h3>THE LAW</h3>')[0]

    #display(HTML(facts))
    h = html2text.HTML2Text()
    facts_raw = h.handle(facts_raw)

    tokens = word_tokenize(facts_raw)

    tokens = [word for word in tokens if word.isalpha()] #remove punctuation
    tokens = [w.lower() for w in tokens] #Lower case

    facts_clean = " ".join(tokens)
    return facts_clean

In [41]:
nonviolation_id = A6_attributes[A6_attributes['target'] == 'nonviolation']['id']
len(nonviolation_id)
nonviolation_id[0:10]

0     HUDOC-ECHR-1958-001-73860
1     HUDOC-ECHR-1961-001-73439
2     HUDOC-ECHR-1962-001-73435
3     HUDOC-ECHR-1964-001-73440
5     HUDOC-ECHR-1966-001-73625
6     HUDOC-ECHR-1968-001-57595
7     HUDOC-ECHR-1969-001-57537
9     HUDOC-ECHR-1970-001-57467
10    HUDOC-ECHR-1970-001-73430
11    HUDOC-ECHR-1970-001-73431
Name: id, dtype: object

In [46]:
ID = list(nonviolation_id)
random_ID = random.sample(ID,1)[0]
print(A6_attributes[A6_attributes['id']==random_ID])
print(clean.getCase(random_ID))
print("+"*50)
print(cleanFacts(random_ID))
print("+"*50)
clean.renderHTML(random_ID)


                               id        target
11374  HUDOC-ECHR-2011-001-106918  nonviolation
{'HUDOC-ECHR-2011-001-106918': '<div>\n<p align="center">FIFTH SECTION</p>\n<p align="center"></p>\n<p align="center"></p>\n<p align="center"><b>CASE OF </b><b>FEXLER v. SWEDEN</b></p>\n<p align="center"><i> (Application no. 36801/06)</i></p>\n<p align="center"></p>\n<p align="center"></p>\n<p align="center">JUDGMENT</p>\n<p align="center"></p>\n<p align="center">STRASBOURG\xa0</p>\n<p align="center"></p>\n<p align="center">13 October 2011</p>\n<p align="center"></p>\n<p align="center"><b><u>FINAL</u></b></p>\n<p align="center"></p>\n<p align="center"><i>13/01/2012</i></p>\n<p align="center"></p>\n<p><i>This judgment has become final under Article 44 § 2 of the Convention. It may be subject to editorial revision</i><i>.</i></p>\n</div>\n<br><div>\n<p><b>In the case of </b>Fexler v. Sweden<b>,</b></p>\n<p>The European Court of Human Rights (Fifth Section), sitting as a Chamber composed of:</p

In [49]:
clean.getCase("HUDOC-ECHR-1958-001-73860")

{'HUDOC-ECHR-1958-001-73860': None}

In [53]:
corpus_nonviolation = []
nv_working_id = []
nv_notworking_id = []

ID = "HUDOC-ECHR-1958-001-73860"
for ID in nonviolation_id:
    try:
        corpus_nonviolation.append(cleanFacts(ID))
        nv_working_id.append(ID)
        print("work") 
    except:
        print("not")
        nv_notworking_id.append(ID)

not
not
not
not
not
not
work
not
not
not
work
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
not
work
not
not
work
work
work
work
not
not
not
work
not
not
work
work
work
not
work
not
work
work
work
n

In [61]:
print(len(corpus_nonviolation),len(nv_working_id),len(nv_notworking_id))
corpus_nonviolation[5]

523 523 440


'circumstances of the case the applicant has the degree of bachelor of laws and is registered as a member of the madrid bar but in fact works as a nurse he submitted that in august one gave him instructions to carry out certain work including in particular a search to discover any encumbrances affecting the title to a piece of land which was among the property made subject to a charging order in summary foreclosure proceedings under the mortgage act sumario ejecutivo de la ley brought by the x company against before judge no of the madrid court of first instance on june bought the land at auction at a price which the applicant considered advantageous the applicant maintained that in he unsuccessfully asked his client to pay him for the administrative services advice and technical assistance he had provided at the time of the purchase in question he asserted that by a registered letter of may with a prepaid form for acknowledgment of receipt to which he received no reply he asked to pay

In [74]:
violation_id = list(A6_attributes[A6_attributes['target'] == 'violation']['id'])
random.shuffle(violation_id)
print(len(violation_id))
violation_id[0:10]

13239


['HUDOC-ECHR-2005-001-71299',
 'HUDOC-ECHR-2000-001-58577',
 'HUDOC-ECHR-1996-001-51358',
 'HUDOC-ECHR-2007-001-83489',
 'HUDOC-ECHR-2011-001-102848',
 'HUDOC-ECHR-1997-001-50606',
 'HUDOC-ECHR-2009-001-92833',
 'HUDOC-ECHR-2005-001-69436',
 'HUDOC-ECHR-1999-001-49993',
 'HUDOC-ECHR-1991-001-57654']

In [75]:
corpus_violation = []
v_working_id = []
v_notworking_id = []

for ID in violation_id:
    try:
        corpus_violation.append(cleanFacts(ID))
        v_working_id.append(ID)
        print("work") 
    except:
        print("not")
        v_notworking_id.append(ID)
    if len(corpus_violation) == 523: break;

work
not
not
not
work
not
work
not
not
not
not
not
not
work
not
not
work
work
not
work
not
work
not
not
not
not
not
not
work
not
not
not
not
not
work
work
work
work
work
work
not
work
not
not
work
not
work
not
not
work
not
work
not
work
not
not
work
not
not
work
not
work
work
not
work
work
not
work
not
work
not
not
work
not
not
not
work
not
not
not
not
work
not
work
not
work
work
not
not
not
not
work
not
not
work
work
work
work
not
not
not
not
work
work
work
work
not
work
work
not
work
work
not
work
not
work
work
not
not
not
work
work
work
work
not
not
not
not
not
not
work
not
work
not
not
not
not
not
work
not
work
work
work
not
work
not
work
work
not
not
not
not
not
work
not
work
not
work
not
work
work
work
not
work
not
not
not
work
work
work
not
work
not
work
work
not
not
work
work
not
not
not
work
not
not
not
not
work
work
work
not
work
work
not
not
not
work
work
not
work
not
not
not
work
work
work
not
not
work
not
work
work
not
not
not
not
work
not
not
not
work
work
work
work
work


In [77]:
print(len(corpus_violation),len(v_working_id),len(v_notworking_id))
corpus_violation[10]

523 523 648


'the circumstances of the case on june a car was broken into and a number of items were stolen shortly thereafter the police apprehended the applicants and took them to a police station there they confessed that they had committed the offence and gave explanations on the manner in which they had acted on the same day the victim of the offence was questioned on july a preliminary investigation was opened against the applicants for the theft of the items from the car on april the preliminary investigation предварително следствие was transformed into a police investigation дознание on february an expert report for assessing the value of the stolen items was commissioned on and february respectively the two applicants were charged and questioned as suspects on october they were questioned by a judge between february and june four witnesses were questioned on april the prosecution authorities filed an indictment with the plovdiv district court and on the the president of the court scheduled

In [80]:
# Create n-gram features 
corpus = corpus_nonviolation + corpus_violation
targets = ['nonviolation']*523 + ['violation']*523
print(len(corpus),len(targets))

1046 1046


In [103]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features= 2000,ngram_range=(1, 4))
X = vectorizer.fit_transform(corpus,y=targets)

In [104]:
X.toarray()

array([[ 2,  2, 16, ..., 10,  1,  0],
       [ 9,  8,  8, ..., 21,  1,  0],
       [ 0,  0,  0, ...,  1,  0,  0],
       ...,
       [ 0,  0,  1, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int64)

In [105]:
features = pd.DataFrame(data = X.toarray(), columns=vectorizer.get_feature_names())


In [106]:
features['target'] = targets
features.head()

,able,able to,about,about the,above,absence,absence of,abuse,accepted,access,...,would,would be,would have,would not,written,year,years,years imprisonment,you,target
0,2,2,16,2,13,5,5,0,4,0,...,35,6,2,3,0,1,10,1,0,nonviolation
1,9,8,8,2,35,1,1,2,4,0,...,47,8,4,10,1,5,21,1,0,nonviolation
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,nonviolation
3,0,0,1,0,2,0,0,0,1,0,...,3,0,2,0,0,1,2,0,0,nonviolation
4,0,0,0,0,1,0,0,0,0,1,...,6,1,1,0,0,0,0,0,0,nonviolation


In [108]:
# Save feature dataset
features.to_csv('/Users/conorosully/Documents/Legal-Case-Prediction/data/test/A6_features.csv')